## Explorative Data Analysis

### Setup

---

In [4]:
%load_ext autoreload
%autoreload 2

In [66]:
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import sys
sys.path.insert(0, '../')

from src import utils

In [73]:
DATA_ROOT = os.path.join("..", "data")
NINAPRO_ROOT = os.path.join("..", "data", "ninapro")
SAMPLING_EMG_RATE = 100

### Load Data and Preprocess Data

---

We are using [NinaPro](http://ninaweb.hevs.ch/) dataset for this project. The dataset is available in the `data` folder. The dataset includes sEMG and kinematic data from 27 intact subjects while repeating 52 hand movements plus the rest position. For the scope of this project, we will only use the sEMG data.

The sEMG data were acquired using 10 Otto Bock MyoBock 13E200 electrodes. The dataset includes 10 repetitions of 52 different movements. The subjects are asked to repeat movements which were shown as movies on the screen of a laptop. The experiment is divided in three exercises:

1. Basic movements of the fingers.
2. Isometric, isotonic hand configurations and basic wrist movements.
3. Grasping and functional movements.

Therefore, for each exercise, for each subject, the database contains one matlab file with synchronized variables. The variables included in the matlab files are:
- Subject: subject number.
- Exercise: exercise number.
- Emg (10 columns): sEMG signal. Columns 1-8 are the electrodes equally spaced around the forearm at the height of the radio humeral joint. Columns 9 and 10 contain signals from the main activity spot of the muscles flexor and extensor digitorum superficialis.
- Stimulus (1 column): the movement repeated by the subject, according to the displayed movie.
- Restimulus (1 column): the movement repeated by the subject, but with the duration of the movement label refined a-posteriori in order to better correspond to the real movement. 
- Repetition (1 column): repetition of the stimulus.
- Rerepetition (1 column): repetition of restimulus.

Let's start with the overview of the subjects:

In [37]:
subjects = pd.read_csv(os.path.join(NINAPRO_ROOT, "subjects.csv"))
subjects.head()

,Subject,Hand,Handedness,Gender,Age,Height,Weight
0,1,Intact,Right,Male,31,170,75
1,2,Intact,Right,Male,27,170,62
2,3,Intact,Right,Male,22,180,85
3,4,Intact,Right,Male,27,183,95
4,5,Intact,Right,Male,27,178,75


Now, we will pefrom some basic data exploration to get a better understanding of the subjects:

In [22]:
male_n, female_n = subjects.Gender.value_counts()
print(f"ℹ️ There are {male_n} males and {female_n} females in the dataset.")

left, right = subjects['Handedness'].value_counts()
print(f"ℹ️ There are {left} left-handed and {right} right-handed subjects in the dataset.")

pd.DataFrame([subjects['Age'].describe(), subjects['Height'].describe(), subjects['Weight'].describe()])

ℹ️ There are 20 males and 7 females in the dataset.
ℹ️ There are 25 left-handed and 2 right-handed subjects in the dataset.


,count,mean,std,min,25%,50%,75%,max
Age,27.0,28.592593,4.235920,22.0,26.5,28.0,30.0,40.0
Height,27.0,173.148148,8.263468,155.0,167.5,173.0,180.0,185.0
Weight,27.0,69.407407,12.530760,48.0,61.0,67.0,75.5,100.0


To make our life easier, we will parse the data for each exercise into a numpy array in a following fashion:

1. For each subject we load the emg data and corresponding stimulus and repetiion labels. These come as 2D arrays where each row corresponds to a timestamp. Therefore, in each row we will have info about the raw emg signal, stimilus, repetition and subject id.

2. We will then concatenate all the subjects into a single numpy array. In other words, we will stack the 2d arrays on top of each other. This way, we can later choose easily which subjects to use for training and which for testing.

If needed in the later stage, we can also merge the data from all three exercises into a single numpy array. But for the start, it is easier to try training the model for recognizing the movements from each exercise separately.

In [61]:
subject_ids = subjects['Subject'].unique().tolist()
exercise_ids = [1, 2, 3]

for ex in exercise_ids:
    print(f"ℹ️ Parsing exercise {ex}")
    exercise = utils.parse_exercise_data(NINAPRO_ROOT, subject_ids, ex)
    print(f"ℹ️ Exercise {ex} has shape {exercise.shape}")

    save_path = os.path.join(DATA_ROOT, f"exercise_{ex}.npy")
    np.save(save_path, exercise)
    print(f"✅ Successfully saved to {save_path}.\n")

ℹ️ Parsing exercise 1


Loading data for exercise 1: 100%|██████████| 27/27 [00:01<00:00, 14.16it/s]


ℹ️ Exercise 1 has shape (2731393, 13)
✅ Successfully saved to ../data/exercise_1.npy.

ℹ️ Parsing exercise 2


Loading data for exercise 2: 100%|██████████| 27/27 [00:03<00:00,  8.86it/s]


ℹ️ Exercise 2 has shape (3868568, 13)
✅ Successfully saved to ../data/exercise_2.npy.

ℹ️ Parsing exercise 3


Loading data for exercise 3: 100%|██████████| 27/27 [00:04<00:00,  6.01it/s]


ℹ️ Exercise 3 has shape (5953650, 13)
✅ Successfully saved to ../data/exercise_3.npy.



Let's check the sizes of the extracted numpy arrays:

In [67]:
!du -h ../data/exercise_1.npy
!du -h ../data/exercise_2.npy
!du -h ../data/exercise_3.npy

602M	../data/exercise_3.npy


392M	../data/exercise_2.npy
280M	../data/exercise_1.npy


### Inspect Exercise Data

---

Start by loading:

In [72]:
exercises = []
for ex in exercise_ids:
    exercise = np.load(os.path.join(DATA_ROOT, f"exercise_{ex}.npy"))
    print(f"✅ Exercise {ex} loaded, it has {exercise.shape[0]} rows and {exercise.shape[1]} columns.")
    exercises.append(exercise)

✅ Exercise 1 loaded, it has 2731393 rows and 13 columns.
✅ Exercise 2 loaded, it has 3868568 rows and 13 columns.
✅ Exercise 3 loaded, it has 5953650 rows and 13 columns.


Then, let's do some basic data exploration:

In [83]:
total_stimuli = 0
for i, exercise in enumerate(exercises):

    # Number of samples per subject
    samples_per_subject = exercise.shape[0] // len(subject_ids)

    # Number of minutes per subject
    minutes_per_subject = samples_per_subject / SAMPLING_EMG_RATE / 60

    # Number of minutes in total
    minutes_total = minutes_per_subject * len(subject_ids)

    # Unique stimulus and repetitions
    stimuli, rep = exercise[:, 10], exercise[:, 11]
    stimuli_unique = np.unique(stimuli)
    total_stimuli += len(stimuli_unique) - 1 # -1 because 0 is not a stimulus but a rest
    rep_unique = np.unique(rep)
    assert (len(rep_unique) - 1) == 10, "❌ There should be 10 repetitions per stimulus." # -1 because 0 is not a repetition but a rest

    print(f"ℹ️ Exercise {i+1} has {len(subject_ids)} subjects, {samples_per_subject} samples per subject, {minutes_per_subject:.2f} minutes per subject, {minutes_total:.2f} minutes in total, {len(stimuli_unique) - 1} unique stimuli and {len(rep_unique)} unique repetitions.")

assert total_stimuli == 52, "❌There should be 52 unique stimuli in total."
print("✅ There are 52 unique stimuli in total.")
print("✅ Each exercise has 10 repetitions per stimulus.")

ℹ️ Exercise 1 has 27 subjects, 101162 samples per subject, 16.86 minutes per subject, 455.23 minutes in total, 12 unique stimuli and 11 unique repetitions.
ℹ️ Exercise 2 has 27 subjects, 143280 samples per subject, 23.88 minutes per subject, 644.76 minutes in total, 17 unique stimuli and 11 unique repetitions.
ℹ️ Exercise 3 has 27 subjects, 220505 samples per subject, 36.75 minutes per subject, 992.27 minutes in total, 23 unique stimuli and 11 unique repetitions.
✅ There are 52 unique stimuli in total.
✅ Each exercise has 10 repetitions per stimulus.
